In [1]:
!pip install -q -U requests pydantic-ai "pydantic-ai-slim[groq]"


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
import requests

BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "219451fef81c8b41e2f6e5a6249db6f2"

def find_weather(city: str) -> dict:
    """This function returns the current weather forecast for the given city"""
    units = "metric"
    params = {
      'q': city,
      'appid': API_KEY,
      'units': units
    }
    response = requests.get(BASE_URL, params=params)
    result = response.json()
    return result

In [6]:
output = find_weather("Nagpur")
print(output)

{'coord': {'lon': 79.1, 'lat': 21.15}, 'weather': [{'id': 721, 'main': 'Haze', 'description': 'haze', 'icon': '50d'}], 'base': 'stations', 'main': {'temp': 30.01, 'feels_like': 28.36, 'temp_min': 30.01, 'temp_max': 30.01, 'pressure': 1012, 'humidity': 23, 'sea_level': 1012, 'grnd_level': 977}, 'visibility': 5000, 'wind': {'speed': 2.57, 'deg': 340}, 'clouds': {'all': 0}, 'dt': 1770461520, 'sys': {'type': 1, 'id': 9069, 'country': 'IN', 'sunrise': 1770427109, 'sunset': 1770467819}, 'timezone': 19800, 'id': 1262180, 'name': 'Nagpur', 'cod': 200}


In [7]:
print(output["name"])
print(output["weather"][0]["description"].capitalize())
print(output["main"]["temp"])

Nagpur
Haze
30.01


In [8]:
import os
import requests
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from pydantic_ai.settings import ModelSettings

In [9]:
os.environ["GROQ_API_KEY"] = "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"

In [10]:
# Define the output schema for the tool
class WeatherForecast(BaseModel):
    location: str
    description: str
    temperature_celsius: float

In [11]:
weather_agent = Agent(
    model="groq:llama-3.1-8b-instant",
    model_settings=ModelSettings(temperature=0.2),
    output_type=str,
    system_prompt=(
        "You are a helpful weather assistant. "
        "Use the 'get_weather_forecast' tool to find current weather conditions "
        "for any city. Provide clean and friendly answers."
    )
)


In [12]:
# weather forecast tool
@weather_agent.tool
def get_weather_forecast(ctx: RunContext, city: str) -> WeatherForecast:
    """
    Returns current weather forecast for a city using OpenWeatherMap API.
    """
    url = "https://api.openweathermap.org/data/2.5/weather"
    api_key = "219451fef81c8b41e2f6e5a6249db6f2"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'
    }
    response = requests.get(url, params=params)
    result = response.json()

    return WeatherForecast(
        location=result["name"],
        description=result["weather"][0]["description"].capitalize(),
        temperature_celsius=result["main"]["temp"]
    )

In [14]:
agent_response = weather_agent.run_sync("What is the weather in Puna")
print(agent_response.output)

It's a clear sky in Puna with a temperature of 30.22°C.


In [19]:
"""
This code takes a user’s weather question and passes it to an AI agent using run_sync.
The agent reasons, calls tools if required, and returns a final response.
"""

question = input("🌤️ Ask about the weather: ")
result = weather_agent.run_sync(question)
print("\n📍 Forecast:", result.output)



📍 Forecast: It seems like the current weather forecast for Mumbai is a smoke-filled sky with a temperature of around 30.99 degrees Celsius.


In [1]:
"""
Weather Assistant using Pydantic-AI and OpenWeatherMap API
This script creates a conversational agent that can respond to weather-related queries
using the OpenWeatherMap API and a Groq-hosted LLaMA model.
"""

# Allow nested async loops (useful for Jupyter or async environments)
import nest_asyncio
nest_asyncio.apply()

# Standard libraries
import os
import requests

# Pydantic model for structured data
from pydantic import BaseModel

# Core AI libraries from pydantic_ai
from pydantic_ai import Agent, RunContext
from pydantic_ai.settings import ModelSettings

os.environ["GROQ_API_KEY"] = "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"

# 1. Define the output schema of the tool using Pydantic
class WeatherForecast(BaseModel):
    location: str
    description: str
    temperature_celsius: float

# 2. Create the AI agent using Groq’s LLaMA 3 model
weather_agent = Agent(
    model="groq:llama-3.1-8b-instant",
    model_settings=ModelSettings(temperature=0.2),
    output_type=str,
    system_prompt=(
        "You are a helpful weather assistant. Use the 'get_weather_forecast' tool to "
        "find current weather conditions for any city. Provide clean and friendly answers."
    )
)

# 3. Register a tool with the agent to fetch real-time weather using OpenWeatherMap API
@weather_agent.tool
def get_weather_forecast(ctx: RunContext, city: str) -> WeatherForecast:
    """
    Tool: get_weather_forecast
    Description: Fetches current weather for a city using the OpenWeatherMap API.
    """
    url = "https://api.openweathermap.org/data/2.5/weather"
    
    # Replace this with your own API key for production use
    api_key = "219451fef81c8b41e2f6e5a6249db6f2"
    
    # Query parameters
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'
    }

    # Send request to weather API
    res = requests.get(url, params=params).json()

    # Return the formatted weather information
    return WeatherForecast(
        location=res["name"],
        description=res["weather"][0]["description"].capitalize(),
        temperature_celsius=res["main"]["temp"]
    )

# 4. Run continuous user interaction loop
if __name__ == "__main__":
    print("🌦️  Weather Assistant is ready! Type 'exit' to quit.")
    print("‒" * 50)
    while True:
        question = input("🌤️ Ask about the weather: ").strip()
        if question.lower() in {"exit", "quit", ""}:
            print("\n👋 Exiting weather assistant. Have a nice day!")
            break

        try:
            result = weather_agent.run_sync(question)
            print("\n📍 Forecast:", result.output)
        except Exception as e:
            print("⚠️ Error:", str(e))

        print("‒" * 50)

🌦️  Weather Assistant is ready! Type 'exit' to quit.
‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒

📍 Forecast: It seems like the weather in Nagpur is currently hazy with a temperature of around 29 degrees Celsius.
‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒
⚠️ Error: status_code: 400, model_name: llama-3.1-8b-instant, body: {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=get_weather_forecast>{"city": "Hyderabad"}'}}
‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒
⚠️ Error: status_code: 400, model_name: llama-3.1-8b-instant, body: {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=get_weather_forecast>{"city": "Puna"}'}}
‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒